In [ ]:
# default_exp leaderboard

# Leaderboard

> Current leaderboard using validation set. The final leaderboard will be generated at the end of the contest using test dataset and will probably be different due to overfitting.

In [ ]:
#hide
in_test = True

In [ ]:
#skip
# ignore this
in_test = False

In [ ]:
#exporti

import pandas as pd
import numpy as np
import boto3
from pathlib import Path
import datetime

In [ ]:
#exporti

import zipfile
import shutil
import torch
import tempfile

from dolphins_recognition_challenge.datasets import get_dataset, get_test_dataset
from dolphins_recognition_challenge.instance_segmentation.model import *

In [ ]:
# exporti


def parse_filename(fname):
    tmp = fname.split("-")
    date = pd.to_datetime(tmp[1] + tmp[2] + tmp[3])
    alias = tmp[6]
    email = tmp[7]
    submitted_iou = tmp[5].split("=")[1]

    return {
        "file_name": fname,
        "date": date,
        "alias": alias,
        "email": email,
        "submitted_iou": submitted_iou,
        "calculated_iou": np.nan,
    }

In [ ]:
# hide

actual = parse_filename(
    "uploaded-2020-12-22T15:35:15.513570-submission-iou=0.46613-dolphin123-name.surname@gmail.com-2020-12-22T15:35:04.875962.zip"
)
expected = {
    "file_name": "uploaded-2020-12-22T15:35:15.513570-submission-iou=0.46613-dolphin123-name.surname@gmail.com-2020-12-22T15:35:04.875962.zip",
    "date": pd.to_datetime("2020-12-22 15:35:15.513570"),
    "alias": "dolphin123",
    "email": "name.surname@gmail.com",
    "submitted_iou": "0.46613",
    "calculated_iou": np.nan,
}

assert actual == expected

In [ ]:
# exporti

s3 = boto3.resource("s3")
my_bucket = s3.Bucket("ai-league.cisex.org")
private_leaderboard_path = Path("private_leaderboard.csv")
public_leaderboard_path = Path("leaderboard.csv")


def get_submissions_from_s3(private_leaderboard_path=private_leaderboard_path):
    """Downloads the zip file from s3 if there is no record of it in the csv file"""
    if private_leaderboard_path.exists():
        private_leaderboard = pd.read_csv(private_leaderboard_path)
    else:
        private_leaderboard = dict(file_name=[])

    # download file into models_for_evaluation directory
    s3_objects = [
        s3_object
        for s3_object in my_bucket.objects.all()
        if Path(s3_object.key).match("*submission*.zip")
        and Path(s3_object.key).name not in list(private_leaderboard["file_name"])
    ]
    if len(s3_objects) > 0:
        for i, s3_object in enumerate(s3_objects):
            print(f"Downloading {i+1}/{len(s3_objects)} from S3...")
            my_bucket.download_file(s3_object.key, f"models_for_evaluation/{Path(s3_object.key).name}")

        # return new entries
        new_entries = pd.Series([Path(s3_object.key).name for s3_object in s3_objects]).apply(parse_filename).apply(pd.Series)
    else:
        x = "uploaded-2020-12-22T15:35:15.513570-submission-iou=0.46613-dolphin123-name.surname@gmail.com-2020-12-22T15:35:04.875962.zip"
        new_entries = pd.Series([x]).apply(parse_filename).apply(pd.Series).iloc[:0, :]
        
    return new_entries
    

In [ ]:
#exporti

def public(private_leaderboard):
    return private_leaderboard[["alias", "date", "submitted_iou", "calculated_iou"]]

In [ ]:
#hide

if not in_test:
    new_entries = get_submissions_from_s3()
    public(new_entries)

In [ ]:
# exporti

def merge_with_private_leaderboard(
    new_entries, private_leaderboard_path=private_leaderboard_path
):
    # merge private leaderboard and new_entries if needed
    new_entries["calculated_iou"] = np.nan
    if private_leaderboard_path.exists():
        private_leaderboard = pd.read_csv(private_leaderboard_path)
        private_leaderboard = pd.concat([private_leaderboard, new_entries], axis=0)
        private_leaderboard = private_leaderboard.drop_duplicates(subset="file_name")
    else:
        private_leaderboard = new_entries

    private_leaderboard.to_csv(private_leaderboard_path, index=False)

    return private_leaderboard

In [ ]:
#hide

if not in_test:
    private_leaderboard = merge_with_private_leaderboard(new_entries)
    public(private_leaderboard)

In [ ]:
# exporti


def evaluate_model(model_path, mode: str = "test") -> float:
    # do it
    with tempfile.TemporaryDirectory() as d:
        with zipfile.ZipFile(model_path, "r") as zip_ref:
            zip_ref.extractall(path=d)
            unzipped_path = [x for x in Path(d).glob("submiss*")][0]
        
        model = torch.load(unzipped_path / "model.pt")
        if mode.lower() == "val":
            _, data_loader = get_dataset("segmentation", batch_size=4)
        elif mode.lower() == "test":
            data_loader = get_test_dataset("segmentation", batch_size=4)
        else:
            raise ValueError()
        iou, iou_df = iou_metric(model, data_loader.dataset)

    return iou

In [ ]:
#hide

if not in_test:
    actual = evaluate_model(
        Path(
            "models_for_evaluation/uploaded-2021-01-05T15:01:23.563795-submission-iou=0.44003-dolphin123-name.surname@gmail.com-2021-01-05T15:01:21.655750.zip"
        ), mode="Val"
    )
    expected = 0.44003
    np.testing.assert_almost_equal(actual, expected, decimal=5)

In [ ]:
datetime.datetime(2021, 6, 5, 0, 0)


datetime.datetime(2021, 6, 5, 0, 0)

In [ ]:
#exporti

cut_off_date = datetime.datetime(2021, 6, 5, 0, 0)

def evaluate_private_leaderboard(private_leaderboard_path=private_leaderboard_path):
    private_leaderboard = pd.read_csv(private_leaderboard_path,parse_dates=["date"])
    private_leaderboard = private_leaderboard[private_leaderboard["date"] < cut_off_date]
    new_entries = private_leaderboard.loc[private_leaderboard["calculated_iou"].isna()]
    
    n = new_entries.shape[0]
    for i, ix in enumerate(new_entries.index):
        row = new_entries.loc[ix]
        file_name, alias, dt = row["file_name"], row["alias"], row["date"]
        print(f"Evaluating model {i+1}/{n} for {alias} submitted at {dt}...")
        calculated_iou = evaluate_model(f"models_for_evaluation/{file_name}", mode="test")
        private_leaderboard.loc[ix, "calculated_iou"] = calculated_iou
        
    private_leaderboard.to_csv(private_leaderboard_path, index=False)
    return private_leaderboard

In [ ]:
#hide

if not in_test:
    evaluate_private_leaderboard()
    private_leaderboard = pd.read_csv("private_leaderboard.csv")
    assert not private_leaderboard["calculated_iou"].isna().any()

Evaluating model 1/136 for stokic submitted at 2021-02-21 18:51:53.232539...
Evaluating model 2/136 for firstML submitted at 2021-02-28 09:54:54.020979...
Evaluating model 3/136 for firstML submitted at 2021-02-28 09:55:59.155951...
Evaluating model 4/136 for rangoiv_0.0 submitted at 2021-03-11 21:18:19.755089...
Evaluating model 5/136 for testy submitted at 2021-03-12 15:06:14.210987...
Evaluating model 6/136 for tekashi submitted at 2021-03-27 12:07:10.438811...
Evaluating model 7/136 for dolphinSantiago submitted at 2021-03-28 21:13:52.740719...
Evaluating model 8/136 for dupincek submitted at 2021-03-29 17:04:06.266327...
Evaluating model 9/136 for Dupin submitted at 2021-03-31 00:17:36.538368...
Evaluating model 10/136 for Dupin submitted at 2021-03-31 13:34:11.751663...
Evaluating model 11/136 for Dupin submitted at 2021-03-31 14:35:22.171495...
Evaluating model 12/136 for tekashi submitted at 2021-03-31 15:40:50.909294...
Evaluating model 13/136 for Dupin submitted at 2021-03-31

Evaluating model 106/136 for duplin submitted at 2021-06-03 09:59:18.678395...
Evaluating model 107/136 for AquamanZo submitted at 2021-06-03 17:04:02.918431...
Evaluating model 108/136 for AquamanZo submitted at 2021-06-03 17:04:34.739131...
Evaluating model 109/136 for AquamanZo submitted at 2021-06-03 17:05:06.111293...
Evaluating model 110/136 for AquamanZo submitted at 2021-06-03 17:05:55.772358...
Evaluating model 111/136 for AquamanZo submitted at 2021-06-03 19:32:37.145129...
Evaluating model 112/136 for AquamanZo submitted at 2021-06-03 20:27:38.962213...
Evaluating model 113/136 for assert0 submitted at 2021-06-03 21:40:16.765718...
Evaluating model 114/136 for assert0 submitted at 2021-06-03 21:54:44.044253...
Evaluating model 115/136 for assert0 submitted at 2021-06-03 22:00:20.069381...
Evaluating model 116/136 for duplin submitted at 2021-06-03 23:51:45.089378...
Evaluating model 117/136 for assert0 submitted at 2021-06-04 08:37:00.543298...
Evaluating model 118/136 for a

In [ ]:
#exporti

def save_public_leaderboard(private_leaderboard_path=private_leaderboard_path, public_leaderboard_path=public_leaderboard_path):
    private_leaderboard = pd.read_csv(private_leaderboard_path)
    public_leaderboard = public(private_leaderboard)
    public_leaderboard.to_csv(public_leaderboard_path, index=False)

In [ ]:
#hide

if not in_test:
    save_public_leaderboard()

In [ ]:
# export


def get_leaderboard(public_leaderboard_path=public_leaderboard_path):
    public_leaderboard = pd.read_csv(public_leaderboard_path)
    public_leaderboard = public_leaderboard[
        (public_leaderboard.alias != "dolphin123")
        & (public_leaderboard.alias != "malimedo")
        & (public_leaderboard.alias != "prvi_pokušaj")
    ]
    public_leaderboard = public_leaderboard.sort_values(
        by=["calculated_iou"], ascending=False
    ).reset_index(drop=True)
    public_leaderboard.drop_duplicates(subset="alias", keep="first", inplace=True)

    public_leaderboard = public_leaderboard.sort_values(
        by=["calculated_iou"], ascending=False
    ).reset_index(drop=True)
    public_leaderboard.index = public_leaderboard.index + 1
    return public_leaderboard

This is a temporary leaderboard calculated daily using validation data. The final leaderboard will be calculated using test dataset unavailable to participants and will most likely be different than the one provided here due to overfitting on the validation dataset. Please see the following link for the details: https://en.wikipedia.org/wiki/Training,_validation,_and_test_sets

In [ ]:
#hide_input

if not in_test:
    display(get_leaderboard())

,alias,date,submitted_iou,calculated_iou
1,AquamanZo,2021-06-03 17:05:55.772358,0.51072,0.516741
2,tekashi,2021-04-20 19:31:43.024354,0.50132,0.513168
3,dolphinSantiago,2021-03-28 21:13:52.740719,0.48156,0.490305
4,alias,2021-06-04 13:05:31.930730,0.57413,0.487422
5,alias1,2021-06-04 10:45:15.451028,0.57750,0.473954
6,duplin,2021-06-03 09:59:18.678395,0.46366,0.471571
7,Fico,2021-06-02 13:18:06.353793,0.46946,0.468637
8,Orka,2021-04-02 16:21:59.548029,0.47683,0.467433
9,Boto,2021-03-31 22:30:51.956628,0.44334,0.461158
10,Dupin,2021-03-31 14:35:22.171495,0.46228,0.457796


In [ ]:
# hide
public_leaderboard_path

PosixPath('leaderboard.csv')